In [ ]:
# import pandas as pd

# df = pd.read_csv("data/animelist-dataset/anime-dataset-2023.csv")

# new_df = df.copy()

# new_df['Korean Name'] = ""
# new_df['Korean Synopsis'] = ""
# new_df.to_csv("data/new-anime-dataset-2023.csv", index=False)

In [22]:
import pandas as pd

df = pd.read_csv("data/animelist-dataset/new-anime-dataset-2023.csv")

In [23]:
df_known = df[df['Score'] != "UNKNOWN"]
df_sorted_score = df_known.sort_values('Score',ascending=False)

In [24]:
df_sorted_score

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL,Korean Name,Korean Synopsis
3961,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.1,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Apr 5, 2009 to Jul 4, 2010",...,24 min per ep,R - 17+ (violence & profanity),1.0,3,217606,2020030.0,3176556,https://cdn.myanimelist.net/images/anime/1208/...,NaN,NaN
16617,41467,Bleach: Sennen Kessen-hen,Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,9.07,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,13.0,"Oct 11, 2022 to Dec 27, 2022",...,24 min per ep,R - 17+ (violence & profanity),2.0,464,17999,213872.0,445198,https://cdn.myanimelist.net/images/anime/1908/...,NaN,NaN
5667,9253,Steins;Gate,Steins;Gate,STEINS;GATE,9.07,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,24.0,"Apr 6, 2011 to Sep 14, 2011",...,24 min per ep,PG-13 - Teens 13 or older,3.0,13,182964,1336233.0,2440369,https://cdn.myanimelist.net/images/anime/1935/...,NaN,NaN
9880,28977,Gintama°,Gintama Season 4,銀魂°,9.06,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,51.0,"Apr 8, 2015 to Mar 30, 2016",...,24 min per ep,PG-13 - Teens 13 or older,4.0,331,15947,237957.0,595767,https://cdn.myanimelist.net/images/anime/3/720...,NaN,NaN
22348,51535,Shingeki no Kyojin: The Final Season - Kankets...,Attack on Titan: The Final Season - Final Chap...,進撃の巨人 The Final Season完結編,9.05,"Action, Drama, Suspense",In the wake of Eren Yeager's cataclysmic actio...,Special,2.0,"Mar 4, 2023 to 2023",...,1 hr 1 min per ep,R - 17+ (violence & profanity),7.0,479,9078,155773.0,435672,https://cdn.myanimelist.net/images/anime/1279/...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10984,31634,Kokuhaku,UNKNOWN,コクハク,2.3,Horror,"Beside a certain building, a girl appears to m...",ONA,1.0,"Aug 27, 2015",...,40 sec,R - 17+ (violence & profanity),12697.0,7105,17,4904.0,6697,https://cdn.myanimelist.net/images/anime/9/762...,NaN,NaN
387,413,Hametsu no Mars,Mars of Destruction,破滅のマルス,2.22,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,1.0,"Jul 6, 2005",...,19 min,R - 17+ (violence & profanity),12698.0,2512,295,47630.0,65622,https://cdn.myanimelist.net/images/anime/7/688...,NaN,NaN
4194,5569,Tsui no Sora,UNKNOWN,終ノ空,2.22,Hentai,"After the sudden death of a student, mysteriou...",OVA,1.0,"Aug 10, 2002",...,23 min,Rx - Hentai,UNKNOWN,7563,24,3436.0,5713,https://cdn.myanimelist.net/images/anime/1179/...,NaN,NaN
6947,13405,Utsu Musume Sayuri,Striking Daughter,打つ娘サユリ,1.98,"Avant Garde, Comedy",Sayuri is a curious creature who lives on her ...,OVA,1.0,2003,...,3 min,R+ - Mild Nudity,12699.0,4492,50,15873.0,20789,https://cdn.myanimelist.net/images/anime/6/375...,NaN,NaN


In [1]:
import pandas as pd

# 1) 데이터 로드
df = pd.read_csv("data/animelist-dataset/new-anime-dataset-2023-with-korean.csv")

# 2) Korean Name / Korean Synopsis 컬럼 없으면 추가 (NaN으로 초기화)
for col in ["Korean Name", "Korean Synopsis"]:
    if col not in df.columns:
        df[col] = pd.NA

# 3) Score가 UNKNOWN이 아닌 행만, 점수 내림차순으로 정렬
df_known = df[df['Score'] != "UNKNOWN"].copy()
df_sorted_score = df_known.sort_values('Score', ascending=False)

# 4) 원본 df에서 사용할 인덱스 리스트 (정렬된 순서대로)
sorted_indices = list(df_sorted_score.index)

len(sorted_indices), df.shape

(15692, (24905, 26))

In [6]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# 🔧 여기만 네가 바꿔 쓰면 됨
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-4.1-mini"           # 필요하면 다른 모델명으로 변경

client = OpenAI(api_key=OPENAI_API_KEY)
def call_translation_api(
    main_name: str,
    other_name: str,
    english_name: str,
    synopsis: str,
) -> dict:
    main_name = main_name or ""
    other_name = other_name or ""
    english_name = english_name or ""
    synopsis = synopsis or ""

    system_prompt = (
        "당신은 일본 애니메이션 정보를 한국어로 현지화하는 도우미입니다. "
        "입력으로 애니의 Main Name, Other Name, English Name과 영어 Synopsis가 주어집니다. "
        "한국에서 통용되는 자연스러운 애니 제목(Korean Name)과 "
        "한국어로 자연스럽게 번역된 시놉시스(Korean Synopsis)를 생성하세요. "
        "반드시 JSON 형식으로만 응답하세요. 키 이름은 'korean_name', 'korean_synopsis' 입니다."
    )

    user_prompt = f"""
다음 애니메이션 정보를 한국어로 정리해 주세요.

[Main Name]
{main_name}

[Other Name]
{other_name}

[English Name]
{english_name}

[Synopsis (EN)]
{synopsis}

출력 형식 (JSON 예시):

{{
  "korean_name": "강철의 연금술사",
  "korean_synopsis": "엘릭 형제가 연금술을 사용해..."
}}
"""

    try:
        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
        )
        content = resp.choices[0].message.content.strip()

        if content.startswith("```"):
            content = content.strip("`")
            lines = content.splitlines()
            if lines and lines[0].strip().lower().startswith("json"):
                content = "\n".join(lines[1:])

        data = json.loads(content)

        korean_name = data.get("korean_name", "")
        korean_synopsis = data.get("korean_synopsis", "")

        return {
            "korean_name": korean_name,
            "korean_synopsis": korean_synopsis,
        }

    except Exception as e:
        print("[API ERROR]", e)
        return {
            "korean_name": "",
            "korean_synopsis": "",
        }

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import math

class KoreanAnnotator:
    def __init__(self, df, sorted_indices, save_path=None):
        self.df = df
        self.sorted_indices = sorted_indices
        self.pos = 0  # sorted_indices 상에서 현재 위치
        self.current_idx = None
        self.save_path = save_path or "data/animelist-dataset/new-anime-dataset-2023-with-korean.csv"
        
        # 위젯들 (미리 만들어두고, 내용만 바꿔가며 재사용)
        self.info_label = widgets.HTML()
        self.other_name_label = widgets.HTML()
        self.synopsis_area = widgets.Textarea(
            description='Synopsis (EN)',
            layout=widgets.Layout(width='100%', height='150px'),
            style={'description_width': '130px'}  # ← 라벨 영역 넓히기
        )

        self.korean_name_input = widgets.Text(
            description='Korean Name',
            layout=widgets.Layout(width='100%'),
            style={'description_width': '130px'}   # ← 여기도
        )

        self.korean_synopsis_input = widgets.Textarea(
            description='Korean Syn',
            layout=widgets.Layout(width='100%', height='200px'),
            style={'description_width': '130px'}   # ← 여기도
        )
        self.save_button = widgets.Button(
            description='저장하고 다음',
            button_style='success'
        )
        self.status_output = widgets.Output()
        
        self.save_button.on_click(self.on_save_click)
        
        self.main_box = widgets.VBox([
            self.info_label,
            self.other_name_label,
            self.synopsis_area,
            self.korean_name_input,
            self.korean_synopsis_input,
            self.save_button,
            self.status_output,
        ])
    
    def find_next_incomplete_index(self):
        """Korean Name 또는 Korean Synopsis 중 하나라도 비어 있으면(NaN or 빈문자) 그 행 인덱스 찾기."""
        while self.pos < len(self.sorted_indices):
            idx = self.sorted_indices[self.pos]
            row = self.df.loc[idx]
            
            kname = row.get("Korean Name")
            ksyn = row.get("Korean Synopsis")
            
            # 문자열이면 공백 제거 후 비어있는지 확인
            kname_empty = (isinstance(kname, str) and kname.strip() == "")
            ksyn_empty = (isinstance(ksyn, str) and ksyn.strip() == "")
            
            # NaN 이거나, 빈 문자열이면 "미완료"
            if pd.isna(kname) or pd.isna(ksyn) or kname_empty or ksyn_empty:
                return idx
            
            self.pos += 1
        
        # 더 이상 미완료 행 없음
        return None
    
    def render_next(self):
        """다음 작업 대상 행을 화면에 렌더링."""
        clear_output(wait=True)
        
        idx = self.find_next_incomplete_index()
        self.current_idx = idx
        
        if idx is None:
            print("✅ 미완료 행이 더 이상 없습니다! 모든 작업 완료.")
            # 마지막에 한 번 전체 저장
            self.df.to_csv(self.save_path, index=False)
            print(f"최종 파일 저장: {self.save_path}")
            return
        
        row = self.df.loc[idx]
        
        # 정보 표시
        score = row.get("Score")
        main_name = row.get("Name")
        other_name = row.get("Other name")
        # 컬럼명이 'English name' / 'English Name' 등일 수 있으니 둘 다 체크
        english_name = (
            row.get("English name")
            if "English name" in self.df.columns
            else row.get("English Name")
        )
        syn = row.get("Synopsis")
        
        self.info_label.value = f"<b>Index:</b> {idx} | <b>Rank:</b> {self.pos+1}/{len(self.sorted_indices)} | <b>Score:</b> {score}"
        self.other_name_label.value = f"<b>Original Name:</b> {main_name} | <b>Other name:</b> {other_name}"
        
        self.synopsis_area.value = syn if isinstance(syn, str) else ""
        
        # API로 자동 제안 받기
        auto = call_translation_api(
            str(main_name or ""),
            str(other_name or ""),
            str(english_name or ""),
            str(syn or ""),
        )
        suggested_kname = auto.get("korean_name", "")
        suggested_ksyn = auto.get("korean_synopsis", "")
        
        # 기존 값이 있으면 그걸 우선 사용 (중간에 저장된 상태에서 재실행하는 경우)
        kname_old = row.get("Korean Name")
        ksyn_old = row.get("Korean Synopsis")
        
        self.korean_name_input.value = (
            kname_old if isinstance(kname_old, str) and kname_old.strip() != "" else suggested_kname
        )
        self.korean_synopsis_input.value = (
            ksyn_old if isinstance(ksyn_old, str) and ksyn_old.strip() != "" else suggested_ksyn
        )
        
        display(self.main_box)
    
    def on_save_click(self, _):
        """저장 버튼 눌렀을 때: df에 반영하고 CSV로 저장 후 다음으로."""
        if self.current_idx is None:
            return
        
        with self.status_output:
            clear_output(wait=True)
            # 값 저장
            self.df.at[self.current_idx, "Korean Name"] = self.korean_name_input.value
            self.df.at[self.current_idx, "Korean Synopsis"] = self.korean_synopsis_input.value
            
            # 중간 저장
            self.df.to_csv(self.save_path, index=False)
            print(f"✅ 저장 완료! index={self.current_idx}, 현재 pos={self.pos+1}")
        
        # 다음 행으로 이동
        self.pos += 1
        self.render_next()

In [5]:
annotator = KoreanAnnotator(
    df=df,
    sorted_indices=sorted_indices,
    save_path="data/animelist-dataset/new-anime-dataset-2023-with-korean.csv"
)

annotator.render_next()

### 그냥 자동화버전

In [9]:
import json
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-4.1-mini"           # 필요하면 다른 모델명으로 변경

client = OpenAI(api_key=OPENAI_API_KEY)

def call_translation_api(
    main_name: str,
    other_name: str,
    english_name: str,
    synopsis: str,
) -> dict:
    main_name = main_name or ""
    other_name = other_name or ""
    english_name = english_name or ""
    synopsis = synopsis or ""

    system_prompt = (
        "당신은 일본 애니메이션 정보를 한국어로 현지화하는 도우미입니다. "
        "입력으로 애니의 Main Name, Other Name, English Name과 영어 Synopsis가 주어집니다. "
        "한국에서 통용되는 자연스러운 애니 제목(Korean Name)과 "
        "한국어로 자연스럽게 번역된 시놉시스(Korean Synopsis)를 생성하세요. "
        "반드시 JSON 형식으로만 응답하세요. 키 이름은 'korean_name', 'korean_synopsis' 입니다."
    )

    user_prompt = f"""
다음 애니메이션 정보를 한국어로 정리해 주세요.

[Main Name]
{main_name}

[Other Name]
{other_name}

[English Name]
{english_name}

[Synopsis (EN)]
{synopsis}

출력 형식 (JSON 예시):

{{
  "korean_name": "강철의 연금술사",
  "korean_synopsis": "엘릭 형제가 연금술을 사용해..."
}}
"""

    try:
        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
        )
        content = resp.choices[0].message.content.strip()

        # ```json ... ``` 형태로 오는 경우 처리
        if content.startswith("```"):
            content = content.strip("`")
            lines = content.splitlines()
            if lines and lines[0].strip().lower().startswith("json"):
                content = "\n".join(lines[1:])

        data = json.loads(content)

        korean_name = data.get("korean_name", "").strip()
        korean_synopsis = data.get("korean_synopsis", "").strip()

        return {
            "korean_name": korean_name,
            "korean_synopsis": korean_synopsis,
        }

    except Exception as e:
        print("[API ERROR]", e)
        return {
            "korean_name": "",
            "korean_synopsis": "",
        }

In [10]:
from tqdm.notebook import tqdm
import pandas as pd
import json

def is_incomplete_like_annotator(kname, ksyn) -> bool:
    """KoreanAnnotator.find_next_incomplete_index()와 동일한 기준."""
    kname_empty = (isinstance(kname, str) and kname.strip() == "")
    ksyn_empty = (isinstance(ksyn, str) and ksyn.strip() == "")
    
    if pd.isna(kname) or pd.isna(ksyn) or kname_empty or ksyn_empty:
        return True
    return False

def auto_fill_korean_from_annotator_logic(
    df: pd.DataFrame,
    sorted_indices,
    save_path: str = "data/animelist-dataset/new-anime-dataset-2023-with-korean.csv",
    save_every: int = 20,
    show_synopsis: bool = False,
):
    total = len(sorted_indices)
    updated_count = 0

    print("=== 자동 번역 (KoreanAnnotator 로직 그대로) 시작 ===")
    print(f"총 대상 행: {total}\n")

    for i, idx in enumerate(tqdm(sorted_indices, desc="Translating anime", unit="anime")):
        row = df.loc[idx]

        # 기존 값
        kname_old = row.get("Korean Name")
        ksyn_old  = row.get("Korean Synopsis")

        # ✅ KoreanAnnotator와 동일한 '미완료' 판정
        if not is_incomplete_like_annotator(kname_old, ksyn_old):
            # 둘 다 채워져 있으면 그대로 둠
            continue

        # 원본 정보
        main_name = row.get("Name") or ""
        other_name = row.get("Other name") or ""
        english_name = (
            row.get("English name")
            if "English name" in df.columns
            else row.get("English Name")
        ) or ""
        syn = row.get("Synopsis") or ""

        # 🔍 번역 API 호출 (suggestion 생성)
        auto = call_translation_api(
            str(main_name or ""),
            str(other_name or ""),
            str(english_name or ""),
            str(syn or ""),
        )
        suggested_kname = auto.get("korean_name", "") or ""
        suggested_ksyn  = auto.get("korean_synopsis", "") or ""

        # ✅ KoreanAnnotator와 동일한 우선순위:
        #    - 기존 값이 문자열이고 비어있지 않으면 그걸 유지
        #    - 아니면 suggestion 사용
        final_kname = (
            kname_old
            if isinstance(kname_old, str) and kname_old.strip() != ""
            else suggested_kname
        )
        final_ksyn = (
            ksyn_old
            if isinstance(ksyn_old, str) and ksyn_old.strip() != ""
            else suggested_ksyn
        )

        # 🔎 로그로 다 보여주기
        print("\n=======================================")
        print(f"[{updated_count+1}] index={idx} | Score={row.get('Score')}")
        print("=== 원본 정보 ===")
        print(f"- Main Name     : {main_name}")
        print(f"- Other Name    : {other_name}")
        print(f"- English Name  : {english_name}")
        if show_synopsis:
            print(f"- Synopsis (EN) : {syn[:250]}{'...' if len(syn) > 250 else ''}")

        print("\n=== 기존 Korean 값 ===")
        print(f"- 기존 Korean Name : {kname_old}")
        print(f"- 기존 Korean Syn  : {ksyn_old[:150] if isinstance(ksyn_old, str) else ksyn_old}")

        print("\n=== 제안 번역(Suggestion) ===")
        print(f"- 제안 Korean Name : {suggested_kname}")
        print(f"- 제안 Korean Syn  : {suggested_ksyn[:150]}{'...' if len(suggested_ksyn) > 150 else ''}")

        print("\n=== 최종 적용 값(Final) ===")
        print(f"➡️ 최종 Korean Name : {final_kname}")
        print(f"➡️ 최종 Korean Syn  : {final_ksyn[:150] if isinstance(final_ksyn, str) else final_ksyn}")
        print("=======================================")

        # df에 최종 값 저장
        df.at[idx, "Korean Name"] = final_kname
        df.at[idx, "Korean Synopsis"] = final_ksyn
        updated_count += 1

        # 중간 저장
        if updated_count % save_every == 0:
            df.to_csv(save_path, index=False)
            print(f"💾 중간 저장 완료 -> {save_path}")

    df.to_csv(save_path, index=False)
    print("\n=== 자동 번역 완료 ===")
    print(f"총 업데이트된 행: {updated_count}")
    print(f"파일 저장 위치: {save_path}")

    return df

In [11]:
save_path = "data/animelist-dataset/new-anime-dataset-2023-with-korean.csv"

df = auto_fill_korean_from_annotator_logic(
    df=df,
    sorted_indices=sorted_indices,
    save_path=save_path,
    save_every=10,
    show_synopsis=False,  # True로 하면 EN 시놉도 같이 찍힘
)

=== 자동 번역 (KoreanAnnotator 로직 그대로) 시작 ===
총 대상 행: 15692



Translating anime:   0%|          | 0/15692 [00:00<?, ?anime/s]


[1] index=10582 | Score=8.39
=== 원본 정보 ===
- Main Name     : Kamisama Hajimemashita: Kako-hen
- Other Name    : 神様はじめました～過去編～
- English Name  : UNKNOWN

=== 기존 Korean 값 ===
- 기존 Korean Name : nan
- 기존 Korean Syn  : nan

=== 제안 번역(Suggestion) ===
- 제안 Korean Name : 신님은じめました: 과거편
- 제안 Korean Syn  : 어느 날 눈 내리는 신사에서 놀던 땅의 신 모모조노 나나미는 자신의 사역신인 여우 요괴 토모에가 쓰러지고 몸에 어두운 문양이 나타나는 것을 목격한다. 오랜 부재 후 토모에의 옛 주인 미카게가 나타나 토모에의 병을 막기 위해 그를 마법의 주머니 거울 속에 넣는다.

미카...

=== 최종 적용 값(Final) ===
➡️ 최종 Korean Name : 신님은じめました: 과거편
➡️ 최종 Korean Syn  : 어느 날 눈 내리는 신사에서 놀던 땅의 신 모모조노 나나미는 자신의 사역신인 여우 요괴 토모에가 쓰러지고 몸에 어두운 문양이 나타나는 것을 목격한다. 오랜 부재 후 토모에의 옛 주인 미카게가 나타나 토모에의 병을 막기 위해 그를 마법의 주머니 거울 속에 넣는다.

미카

[2] index=13444 | Score=8.39
=== 원본 정보 ===
- Main Name     : Natsume Yuujinchou Roku Specials
- Other Name    : 夏目友人帳 陸 特別編
- English Name  : Natsume's Book of Friends Season 6 Specials

=== 기존 Korean 값 ===
- 기존 Korean Name : nan
- 기존 Korean Syn  : nan

=== 제안 번역(Suggestion) ===
- 제안 Korean Name : 나츠메 우인장 6기 스페셜
- 제

KeyboardInterrupt: 